In [1]:
import os
import openpyxl
import locale
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
from sqlalchemy import create_engine, exc

load_dotenv()

path_dados = os.getenv('DUSNEI_DATA_DIRECTORY_RELATORIO_OUTROS')

In [2]:
def get_db_engine():
    try:
        db_url = os.getenv('DUSNEI_URL')
        engine = create_engine(db_url)
        # Test connection
        with engine.connect() as connection:
            print('Conexão estabelecida!')
            pass
        print('Banco de dados conectado!')
        return engine
    except exc.SQLAlchemyError as e:
        print(f"Error: {e}")
        return None
        

In [45]:
def query_dados(conn):  
    query = (f"""
        (
            SELECT 
                pfin.pfin_datavcto AS datavcto, 
                pfin.pfin_unid_codigo AS unid_cod, 
                pfin.pfin_vend_codigo AS vend_cod, 
                pfin.pfin_codentidade AS entidade_cod, 
                pfin.pfin_cnpjcpf AS cnpj, 
                pfin.pfin_numerodcto AS nota, 
                pfin.pfin_valor AS valor, 
                pger.pger_descricao AS descricao, 
                clie.clie_nome AS clie_nome, 
                clie.clie_razaosocial AS razaosocial,
                clie.clie_foneres AS fone_res,
                clie.clie_fonecel AS fone_cel,
                vend.vend_nome AS vend_nome, 
                vend.vend_supe_codigo AS supe_cod,
                CONCAT(vend.vend_extra16, vend.vend_extra15) AS vend_cel,
                supe.supe_nome AS supe_nome
                FROM pendfin AS pfin
                LEFT JOIN planoger AS pger ON pfin.pfin_pger_conta = pger.pger_conta
                LEFT JOIN clientes AS clie ON pfin.pfin_codentidade = clie.clie_codigo
                LEFT JOIN vendedores AS vend ON clie.clie_vend_codigo = vend.vend_codigo
                LEFT JOIN supervisores AS supe ON vend.vend_supe_codigo = supe.supe_codigo
                WHERE pfin.pfin_status = 'P'
                AND pfin.pfin_datavcto < CURRENT_DATE
                AND pfin.pfin_pger_conta IN ('112131','112140','112201','112202','112203')
                ORDER BY pfin.pfin_datavcto DESC
        )  
    """)
    
    df = pd.read_sql_query(query, conn)
       
    return df

In [46]:
def gerar_excel(df, ws):
    df_resumo = df.groupby(['cod_fpgt','fpgt_nome'], as_index=False)['valor_liq'].sum()

    ws.title = f"F.Pgto"
  
    columns_title = [
        'CodPgto','Descrição','Valor'
    ]
    
    for col_num, column_title in enumerate(columns_title, 1):
        ws.cell(row=1, column=col_num).value = column_title

    for index, row in enumerate(df_resumo.itertuples(), start=2):
        ws.cell(row=index, column=1).value = row.cod_fpgt
        ws.cell(row=index, column=2).value = row.fpgt_nome
        ws.cell(row=index, column=3).value = row.valor_liq
        ws.cell(row=index, column=3).number_format = 'R$ #,##0.00'


In [47]:
def teste_loop():
    conn = get_db_engine()
       
    locale.setlocale(locale.LC_MONETARY, 'pt-BR.UTF-8')

    tables = [
        'movprodd0123', 'movprodd0223', 'movprodd0323', 'movprodd0423', 'movprodd0523', 'movprodd0623', 
        'movprodd0723', 'movprodd0823', 'movprodd0923', 'movprodd1023', 'movprodd1123', 'movprodd1223'
    ]
    wb = openpyxl.Workbook()
    
    for table in tables:
        df = query_dados(conn, table)

        ws = wb.create_sheet(title=table)
        ws.append(['CodPgto', 'NomePgto', 'ValorTotal'])

        for row in df.itertuples(index=False):
            ws.append([row.cod_fpgt, row.fpgt_nome, row.valor_liq])
            ws.cell(row=ws.max_row, column=3).number_format = 'R$ #,##0.00'
    
    if 'Sheet' in wb.sheetnames:
        wb.remove(wb['Sheet'])
    
    data_pasta = datetime.now().strftime("%Y-%m-%d")
    nome_arquivo = f'F.Pgto-{data_pasta}.xlsx'
    diretorio = os.path.join(path_dados, 'outros', data_pasta)
    if not os.path.exists(diretorio):
        os.mkdir(diretorio)
    local_arquivo = os.path.join(diretorio, nome_arquivo)
    wb.save(local_arquivo)

teste_loop()

Conexão estabelecida!
Banco de dados conectado!
